In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import datetime

In [ ]:
from autumn.tools.runs import ManagedRun

from autumn.tools.runs.calibration.utils import get_posteriors
from autumn.tools.utils.pandas import pdfilt

In [ ]:
run_ids, mrs, mcmc_runs, pbis, results, targets = {}, {}, {}, {}, {}, {}
run_ids["west-metro"] = "covid_19/west-metro/1635166421/153a938"
run_ids["barwon-south-west"] = "covid_19/barwon-south-west/1635203341/e17f4a0"
run_ids["gippsland"] = "covid_19/gippsland/1635166443/153a938"
run_ids["grampians"] = "covid_19/grampians/1635166473/153a938"
run_ids["hume"] = "covid_19/hume/1635166677/153a938"
run_ids["loddon-mallee"] = "covid_19/loddon-mallee/1635166635/153a938"
run_ids["north-east-metro"] = "covid_19/north-east-metro/1635204422/94ea65b"
run_ids["south-east-metro"] = "covid_19/south-east-metro/1635166421/153a938"

In [ ]:
regions = ("west-metro", "barwon-south-west", "gippsland", "grampians", "hume", "loddon-mallee", "north-east-metro", "south-east-metro")
for region in regions:
    mrs[region] = ManagedRun(run_ids[region])
    mcmc_runs[region] = mrs[region].calibration.get_mcmc_runs()
    pbis[region] = mrs[region].powerbi.get_db()
    results[region] = pbis[region].get_uncertainty()
    targets[region] = pbis[region].get_targets()

In [ ]:
outputs = ("notifications",)

plot_left_date = datetime.date(2021, 8, 2)



for scenario in [0, 1]:
    fig, axes = plt.subplots(4, 2, figsize=(12, 20), sharex="all")

    for i_ax, axis in enumerate(axes.reshape(-1)[:8]):

        output = "notifications"
        region = regions[i_ax]

        colour = (0.2, 0.2, 0.8)

        axis.fill_between(results[region].index, results[region][(output, scenario, 0.01)], results[region][(output, scenario, 0.99)], color=colour, alpha=0.1)
        axis.fill_between(results[region].index, results[region][(output, scenario, 0.025)], results[region][(output, scenario, 0.975)], color=colour, alpha=0.5)
        axis.fill_between(results[region].index, results[region][(output, scenario, 0.25)], results[region][(output, scenario, 0.75)], color=colour, alpha=0.7)
        axis.plot(results[region].index, results[region][(output, scenario, 0.500)], color="b", linestyle="--")

        axis.scatter(targets[region].index, targets[region][output], color="k", alpha=0.5, s=10)
        axis.tick_params(axis="x", labelrotation=45)
        axis.set_xlim(left=plot_left_date)
        axis.set_title(region)